In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import scipy.io
import gc
import itertools
from sklearn.metrics import confusion_matrix
import sys
sys.path.insert(0, './preparation')
import os

# Keras imports
import keras
from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Flatten, Dropout,MaxPooling1D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import Callback,warnings, ReduceLROnPlateau

In [25]:
path = '/home/ubuntu/projects/ecg/'
data_path = path + 'data/train/training/'

In [26]:
print("Loading data training set")        
matfile = scipy.io.loadmat(data_path+'TrainData1000.mat')

Loading data training set


In [27]:
matfile

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Dec 08 13:48:58 2017',
 '__version__': '1.0',
 'trainData': array([[-59.99505362, -59.98524582, -59.97697805, ..., -59.22539085,
         -59.11628773, -58.42976453],
        [-59.99608979, -59.98861609, -59.98153551, ..., -56.95266751,
         -56.11547729, -55.55837516],
        [-59.99511224, -59.98552185, -59.972777  , ..., -45.71653897,
         -44.46837574, -43.18280277],
        ..., 
        [-59.9410233 , -59.81968743, -59.69713992, ..., -60.74308101,
         -60.59376259, -60.60749349],
        [-59.93932438, -59.81435612, -59.68827016, ..., -60.77285387,
         -60.77969118, -60.58663034],
        [-59.93985226, -59.81825768, -59.69636081, ..., -63.44466112,
         -63.37986803, -63.34375053]]),
 'trainLabels': array([[1, 0],
        [1, 0],
        [1, 0],
        ..., 
        [1, 0],
        [1, 0],
        [1, 0]], dtype=uint8)}

In [28]:
def loaddata(WINDOW_SIZE):    
    '''
        Load training/test data into workspace
        
        This function assumes you have downloaded and padded/truncated the 
        training set into a local file named "trainingset.mat". This file should 
        contain the following structures:
            - trainset: NxM matrix of N ECG segments with length M
            - traintarget: Nx4 matrix of coded labels where each column contains
            one in case it matches ['A', 'N', 'O', '~'].
        
    '''
    print("Loading data training set")        
    matfile = scipy.io.loadmat(data_path+'TrainData13000.mat')
    X = matfile['trainData']
    y = matfile['trainLabels']
    
    # Merging datasets    
    # Case other sets are available, load them then concatenate
    #y = np.concatenate((traintarget,augtarget),axis=0)     
    #X = np.concatenate((trainset,augset),axis=0)     

    X =  X[:,0:WINDOW_SIZE] 
    return (X, y)

In [29]:
def ResNet_model(WINDOW_SIZE):
    #parameters based on from Rajpurkar et al
    INPUT_FEAT = 1
    OUTPUT_CLASS = 2 #normal/abnormal: 

    k = 1
    p = True #pooling alternates every block
    convfilt = 64
    convstr = 1
    kern_size = 16
    poolsize = 2
    poolstr  = 2
    drop = 0.9
    
    input1 = Input(shape=(WINDOW_SIZE,INPUT_FEAT))
    
    #initial convblock
    x = Conv1D(filters=convfilt,
               kernel_size=kern_size,
               padding='same',
               strides=convstr,
               kernel_initializer='he_normal')(input1)                
    x = BatchNormalization()(x)        
    x = Activation('relu')(x)  
    
    #convblock 2 and branches:
    #main sequence
    x1 =  Conv1D(filters=convfilt,
               kernel_size=kern_size,
               padding='same',
               strides=convstr,
               kernel_initializer='he_normal')(x)      
    x1 = BatchNormalization()(x1)    
    x1 = Activation('relu')(x1)
    x1 = Dropout(drop)(x1)
    x1 =  Conv1D(filters=convfilt,
               kernel_size=kern_size,
               padding='same',
               strides=convstr,
               kernel_initializer='he_normal')(x1)
    x1 = MaxPooling1D(pool_size=poolsize,
                      strides=poolstr)(x1)
    #shortcut
    x2 = MaxPooling1D(pool_size=poolsize,
                      strides=poolstr)(x)
    #combine both
    x = keras.layers.add([x1, x2])
    del x1,x2
    
    #repeated convblock creation
    p = not p 
    for l in range(5):
        
        if (l%4 == 0) and (l>0):
            k += 1
            xshort = Conv1D(filters=convfilt*k,kernel_size=1)(x)
        else:
            xshort = x        
        #main section       
        x1 = BatchNormalization()(x)
        x1 = Activation('relu')(x1)
        x1 = Dropout(drop)(x1)
        x1 =  Conv1D(filters=convfilt*k,
               kernel_size=kern_size,
               padding='same',
               strides=convstr,
               kernel_initializer='he_normal')(x1)        
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
        x1 = Dropout(drop)(x1)
        x1 =  Conv1D(filters=convfilt*k,
               kernel_size=kern_size,
               padding='same',
               strides=convstr,
               kernel_initializer='he_normal')(x1)        
        if p:
            x1 = MaxPooling1D(pool_size=poolsize,strides=poolstr)(x1)                

        #shortcut for ResNet
        if p:
            x2 = MaxPooling1D(pool_size=poolsize,strides=poolstr)(xshort)
        else:
            x2 = xshort       
        # combine sides
        x = keras.layers.add([x1, x2])
        #to pool or not to pool
        p = not p 

    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    x = Flatten()(x)
    out = Dense(OUTPUT_CLASS, activation='softmax')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [30]:
#config = tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)
seed = 7
np.random.seed(seed)

# Parameters
FS = 300
WINDOW_SIZE = 30*FS     # padding window for CNN

# Loading data
(X,y) = loaddata(WINDOW_SIZE)

Loading data training set


In [31]:
X.shape

(13000, 9000)

In [32]:
calls = [
                # Early stopping definition
                EarlyStopping(monitor='val_loss', patience=3, verbose=1),
                # Decrease learning rate by 0.1 factor
                ReduceLROnPlateau(monitor='val_loss', patience=1,verbose=1, mode='min', factor=0.1),            
                # Saving best model
                ModelCheckpoint('weights-best_2.hdf5', monitor='val_loss', save_best_only=True, verbose=1),
                ]

In [33]:
batch =64
epochs = 20
Ntrain = X.shape[0] # number of recordings on training set
num_valid = int(Ntrain/5) # number of recordings to take as validation        

In [34]:
# Need to add dimension for training
X = np.expand_dims(X, axis=2)
#classes = ['A', 'N', 'O', '~']
classes = ['N','O']
Nclass = len(classes)

In [35]:
X.shape

(13000, 9000, 1)

In [36]:
X[0].shape

(9000, 1)

In [37]:
# Load model
model = ResNet_model(WINDOW_SIZE)

In [38]:
# split train and validation sets
idxval = np.random.choice(Ntrain, num_valid,replace=False)
idxtrain = np.invert(np.in1d(range(Ntrain),idxval))
ytrain = y[np.asarray(idxtrain),:]
Xtrain = X[np.asarray(idxtrain),:,:]         
Xval = X[np.asarray(idxval),:,:]
yval = y[np.asarray(idxval),:]

In [39]:
Xval.shape

(2600, 9000, 1)

In [40]:
# Train model
model.fit(Xtrain, ytrain,
          validation_data=(Xval, yval),
          epochs=epochs, batch_size=batch,
         callbacks=calls)

Train on 10400 samples, validate on 2600 samples
Epoch 1/20
10400/10400 [==============================] - 266s 26ms/step - loss: 5.5690 - acc: 0.6516 - val_loss: 5.5359 - val_acc: 0.6565
Epoch 2/20
10400/10400 [==============================] - 263s 25ms/step - loss: 5.5917 - acc: 0.6531 - val_loss: 5.5359 - val_acc: 0.6565
Epoch 3/20
10368/10400 [============================>.] - ETA: 0s - loss: 5.5966 - acc: 0.6528
Epoch 00003: reducing learning rate to 0.00010000000475.
Epoch 00003: val_loss did not improve
10400/10400 [==============================] - 264s 25ms/step - loss: 5.5917 - acc: 0.6531 - val_loss: 5.5359 - val_acc: 0.6565
Epoch 4/20
10368/10400 [============================>.] - ETA: 0s - loss: 5.5919 - acc: 0.6531
Epoch 00004: reducing learning rate to 1.0000000475e-05.
Epoch 00004: val_loss did not improve
10400/10400 [==============================] - 263s 25ms/step - loss: 5.5917 - acc: 0.6531 - val_loss: 5.5359 - val_acc: 0.6565
Epoch 00004: early stopping
